In [ ]:
import numpy as np

class IsentropicFlowTable:
    def __init__(self, gamma, maxMach, machResolution):
        self.maxMach = maxMach
        self.gamma = gamma
        self.machResolution = machResolution
        
        


That defines the Isentropic flow table


